# Данные университетов в ИС [rosrid.ru](https://rosrid.ru/)

Основная задача данного отчёта продемонстрировать каким образом можно взять данные по своему университету из информационной системы (сайта) [rosrid.ru](https://rosrid.ru/).

Все механики будут показаны на примере одного университета — [Дальневосточный федеральный университет](https://t.me/psalgroup/10182).

Каждый университет в рамках ИС имеет уникальный ID, по которому и будет осуществляться поиск по всем возможным объектам. На gif-ке ниже показано, каким образом можно узнать ID своего университета (все манипуляции происходят на странице [«Глобального поиска»](https://rosrid.ru/global-search)).

![](assets/get_university_id.gif)

Зафиксируем данный параметр для ДВФУ, а также заведём переменные, которые ограничат нас текущим годом для поиска.

In [1]:
uid = '9944117'
start_date = '2022-01-01'
end_date = '2022-12-31'

Сначала воспроизведём основное, что нам понадобится в работе.

In [2]:
import requests

# URL API, который осуществляет поиск по ИС
search_url = 'https://rosrid.ru/api/base/search'
# Формируем заготовку для основного тела поиска
payload = {
    "search_query": None,
    "critical_technologies": [],
    "dissertations": False,
    "full_text_available": False,
    "ikrbses": False,
    "nioktrs": False,
    "organization": [uid],
    "page": 1,
    "priority_directions": [],
    "rids": False,
    "rubrics": [],
    "search_area": "Во всех полях",
    "sort_by": "Дата регистрации",
    "open_license": False,
    "free_licenses": False,
    "expert_estimation_exist": False,
    "start_date": start_date,
    "end_date": end_date
}

# Создаём сессию и проставляем стандартные заголовки, что бы не сильно отличаться от браузера
session = requests.session()
session.headers.update({
    'authority': 'rosrid.ru',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
    'accept': 'application/json, text/plain, */*',
    'sec-ch-ua-mobile': '?0',
    'content-type': 'application/json;charset=UTF-8',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    'sec-ch-ua-platform': '"Linux"',
    'origin': 'https://rosrid.ru',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://rosrid.ru/global-search',
    'accept-language': 'ru,en-US;q=0.9,en;q=0.8,ru-RU;q=0.7'
})

# Для проверки подключаемся к странице поиска и возвращаем код ответа (должен быть 200)
home_resp = session.request('GET', 'https://rosrid.ru/global-search')
home_resp.status_code

200

Т.к. результаты поиска выдаются в постраничном виде, реализуем функцию, которая позволит нам рекурсивно собрать весь объём данных.

In [3]:
import time
import json

def get_search_results(data, timeout=1):
    items_in_page = 10
    search_results = []

    try:
        resp = session.request("POST", search_url, data=json.dumps(data, ensure_ascii=False).encode('utf-8'))

        if resp.status_code == 200:
            json_resp = resp.json()
            page = data['page']
            total = json_resp['hits']['total']['value']
            count_of_pages = (int(total / items_in_page) + 1) if total % items_in_page else total / items_in_page

            print(f"Downloaded data from page {page} of {count_of_pages}")

            if page < count_of_pages:
                time.sleep(timeout)
                search_results += json_resp['hits']['hits'] + get_search_results({**data, 'page': page + 1}, timeout)
            else:
                search_results += json_resp['hits']['hits']
    except BaseException as e:
        print('Retry connection', str(e))
        search_results = get_search_results({**data, 'page': 1}, timeout)

    return search_results

## НИОКТР

Для начала заберём данные о НИОКТР-ах ДВФУ за 2022 год и посмотрим 5 первых записей.

In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)

nioktrs = pd.json_normalize(get_search_results({**payload, 'nioktrs': True}))
nioktrs.dropna(axis=1).head(5)

Downloaded data from page 1 of 2
Downloaded data from page 2 of 2


,_index,_type,_id,_score,sort,_source.name,_source.annotation,_source.keyword_list,_source.work_supervisor.name,_source.work_supervisor.surname,_source.work_supervisor.patronymic,_source.work_supervisor.position,_source.organization_supervisor.name,_source.organization_supervisor.surname,_source.organization_supervisor.patronymic,_source.organization_supervisor.position,_source.executor.id,_source.executor.organization_id,_source.executor.name,_source.executor.short_name,_source.executor.ogrn,_source.executor.region.name,_source.executor.region.code,_source.executor.okogu.name,_source.coexecutors,_source.customer.id,_source.customer.organization_id,_source.customer.name,_source.customer.short_name,_source.customer.ogrn,_source.nioktr_bases.name,_source.nioktr_types,_source.priority_directions,_source.critical_technologies,_source.budgets,_source.rubrics,_source.oecds,_source.last_status.created_date,_source.last_status.registration_number,_source.number_of_related_rid,_source.start_date,_source.end_date,_source.contract_number,_source.contract_date,_source.reports_number,_source.udk,_source.number_of_views
0,nioktrs,_doc,ZJP5EE4LCTKBP7PH060ZJJWR,0.0,"[1661411639597, 0.0]",Специализированные пищевые системы для профила...,На основании полученных результатов будут спро...,"[{'name': 'восприятие потребителей'}, {'name':...",Антон,Табакаев,Вадимович,Доцент Департамента пищевых наук и технологий ...,Александр,Самардак,Сергеевич,Проректор по научной работе,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,Приморский край,25,Министерство науки и высшего образования Росси...,[],107775997263010000000001,1e88d205da93af783d0d9da,Российский научный фонд,РНФ,1137799022261,Грант,[{'name': 'Фундаментальное исследование'}],[{'name': 'Рациональное природопользование'}],[{'name': 'Биомедицинские и ветеринарные техно...,[{'budget_type': {'name': 'Средства фондов под...,[{'name': 'Отходы производства и их переработк...,"[{'name': 'NUTRITION & DIETETICS', 'code': '03...",2022-08-25T07:13:59.597600+00:00,122082500021-5,0,2022-07-27,2024-06-30,22-76-00008,2022-07-27,2,664:502.171;664:658.567,0
1,nioktrs,_doc,ZG7FZFSFZ5JU6Q1PLVWMVGXB,0.0,"[1661411399139, 0.0]",Системы на основе биополимеров для доставки ле...,"Большинство препаратов, разрабатываемых для те...","[{'name': 'наночастицы'}, {'name': 'биополимер...",Владимир,Силантьев,Евгеньевич,Заведующий кафедрой,Александр,Самардак,Сергеевич,Проректор по научной работе,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,Приморский край,25,Министерство науки и высшего образования Росси...,[],107775997263010000000001,1e88d205da93af783d0d9da,Российский научный фонд,РНФ,1137799022261,Грант,[{'name': 'Фундаментальное исследование'}],[{'name': 'Индустрия наносистем'}],[{'name': 'Технологии получения и обработки ко...,[{'budget_type': {'name': 'Средства фондов под...,"[{'name': 'Структура и функции биополимеров', ...","[{'name': 'NANOSCIENCE & NANOTECHNOLOGY', 'cod...",2022-08-25T07:09:59.139698+00:00,122082500018-5,0,2022-07-29,2025-06-30,22-73-10172,2022-07-29,3,577.1 577.15,1
2,nioktrs,_doc,C6ULS4ADTHJQ83675ME8CWEI,0.0,"[1660896626635, 0.0]",Исследование природных и антропогенных воздейс...,Цель научного исследования – Оценка природных ...,"[{'name': 'природные воздействия'}, {'name': '...",Кирилл,Винников,Андреевич,"Директор Института Мирового океана (школы), за...",Александр,Самардак,Сергеевич,Проректор по научной работе,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,Приморский край,25,Министерство науки и высшего образования Росси...,[],089Y0A0LoDLV15JGaQ2cLb00,5f7f75ebf4703b53b50b78d,МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ РОССИ...,МИНОБРНАУКИ РОССИИ,118774657

## РИД

Теперь заберём данные о РИД-ах ДВФУ за 2022 год и посмотрим 5 первых записей.

In [5]:
rids = pd.json_normalize(get_search_results({**payload, 'rids': True}))
rids.dropna(axis=1).head(5)

Downloaded data from page 1 of 3
Downloaded data from page 2 of 3
Downloaded data from page 3 of 3


,_index,_type,_id,_score,sort,_source.name,_source.keyword_list,_source.abstract,_source.customer.id,_source.customer.organization_id,_source.customer.name,_source.customer.short_name,_source.customer.ogrn,_source.customer.inn,_source.rid_type.name,_source.expected.name,_source.executors,_source.authors,_source.work_supervisor.name,_source.work_supervisor.surname,_source.work_supervisor.patronymic,_source.work_supervisor.position,_source.organization_supervisor.name,_source.organization_supervisor.surname,_source.organization_supervisor.patronymic,_source.organization_supervisor.position,_source.oecds,_source.rubrics,_source.last_status.created_date,_source.last_status.registration_number,_source.additional_files,_source.iksi,_source.ikspo,_source.rid_own_using_exists,_source.rid_external_using_exists,_source.used_rids,_source.using_ways,_source.number_of_prototypes,_source.number_of_views
0,rids,_doc,78WN77ITQGWNX440GF9HB1RZ,0.0,"[1660902576886, 0.0]",Нейронная сеть для поиска основных состояний ...,"[{'name': 'нейронная сеть'}, {'name': 'модели ...",Программа предназначена для поиска конфигураци...,107775997263010000000001,1e88d205da93af783d0d9da,Российский научный фонд,РНФ,1137799022261,7709473426,Программа для ЭВМ,Исполнитель,"[{'id': 'JXDYN5VUB7LATD8HZ4740PRS', 'organizat...","[{'name': 'Егор', 'surname': 'Васильев', 'patr...",Виталий,Капитан,Юрьевич,Доцент,Александр,Самардак,Сергеевич,Проректор по научной работе,"[{'name': 'PHYSICS, ATOMIC, MOLECULAR & CHEMIC...",[{'name': 'Автоматизация физического экспериме...,2022-08-19T09:49:36.886357+00:00,622081900049-3,True,[],[],False,False,[],Научные исследования в области теории конденси...,5,0
1,rids,_doc,K2KTVRPU87TGWNU47PU3I0M5,0.0,"[1659515854705, 0.0]",Программа для трехмерного сканирования объекто...,[{'name': 'протяженные объекты сложной формы'...,Программа для трехмерного сканирования объекто...,102250051035010000000001,ae07dddb6331eec7121e4fa,"Открытое акционерное общество ""Арсеньевская ав...","ОАО ААК ""ПРОГРЕСС""",1022500510350,2501002394,Программа для ЭВМ,Исполнитель,"[{'id': 'JXDYN5VUB7LATD8HZ4740PRS', 'organizat...","[{'name': 'Михаил', 'surname': 'Бабич', 'patro...",Константин,Змеу,Витальевич,Директор департамента,Александр,Самардак,Сергеевич,Проректор по научной работе,"[{'name': 'ENGINEERING, AEROSPACE', 'code': '0...",[{'name': 'Применение роботов и манипуляторов'...,2022-08-03T08:37:34.705887+00:00,622080300025-6,False,[],[{'last_status': {'created_date': '2022-08-19T...,False,False,[],Промышленное 3d- сканирование,5,0
2,rids,_doc,B3DXA8MZF9CXPB3ZY792W0R0,0.0,"[1659515837389, 0.0]",Программа моделирования деталей гидрораспредел...,"[{'name': '3d модель'}, {'name': 'компьютерное...",Высокопроизводительная программа для ЭВМ предн...,102250051035010000000001,ae07dddb6331eec7121e4fa,"Открытое акционерное общество ""Арсеньевская ав...","ОАО ААК ""ПРОГРЕСС""",1022500510350,2501002394,Программа для ЭВМ,Исполнитель,"[{'id': 'JXDYN5VUB7LATD8HZ4740PRS', 'organizat...","[{'name': 'Сергей', 'surname': 'Гречанов', 'pa...",Константин,Змеу,Витальевич,Директор департамента,Александр,Самардак,Сергеевич,Проректор по научной работе,"[{'name': 'ENGINEERING, AEROSPACE', 'code': '0...","[{'name': 'Технология авиастроения', 'code': '...",2022-08-03T08:37:17.389516+00:00,622080300024-9,False,[],[{'last_status': {'created_date': '2022-08-19T...,False,False,[],машиностроение и судостроение,6,0
3,rids,_doc,IA00NIZJSITLLHS72XKNPRLS,0.0,"[1659417918600, 0.0]",Программа для визуализации результатов работы ...,"[{'name': 'визуализация результатов работы'}, ...",Программа для визуализации результатов работы ...,102250051035010000000001,ae07dddb6331eec7121e4fa,"Открытое акционерное общество ""Арсеньевская ав...","ОАО ААК ""ПРОГРЕСС""",1022500510350,2501002394,Программа для ЭВМ,Исполнитель,"[{'id': 'JXDYN5VUB7LATD8HZ4740PRS', 'organizat...","[{'name': 'Михаил', 'surname': 'Бабич', 'patro...",Константин,Змеу,Витальевич,Директор департамента,Александр,Самардак,Сергеевич,Проректор по научной ра

## Диссертации

Последний набор данных будет содержать информацию о диссертациях связанных с ДВФУ.

In [6]:
dissertations = pd.json_normalize(get_search_results({**payload, 'dissertations': True}))
dissertations.dropna(axis=1).head(5)

Downloaded data from page 1 of 1


,_index,_type,_id,_score,sort,_source.name,_source.keyword_list,_source.abstract,_source.dissertation_type.name,_source.dissertation_report_type.name,_source.degree_pursued.name,_source.protection_organization.id,_source.protection_organization.organization_id,_source.protection_organization.name,_source.protection_organization.short_name,_source.protection_organization.ogrn,_source.protection_organization.inn,_source.protection_organization.region.name,_source.protection_organization.region.code,_source.author_organization.id,_source.author_organization.organization_id,_source.author_organization.name,_source.author_organization.short_name,_source.author_organization.ogrn,_source.author_organization.inn,_source.author_organization.region.name,_source.author_organization.region.code,_source.chairman_dissertation_council.name,_source.chairman_dissertation_council.surname,_source.chairman_dissertation_council.patronymic,_source.chairman_dissertation_council.position,_source.chairman_dissertation_council.degree.name,_source.organization_supervisor.name,_source.organization_supervisor.surname,_source.organization_supervisor.patronymic,_source.organization_supervisor.position,_source.speciality_codes,_source.rubrics,_source.oecds,_source.last_status.created_date,_source.last_status.registration_number,_source.supervisors,_source.opponents,_source.protection_date,_source.author_surname,_source.author_name,_source.author_patronymic,_source.tables_count,_source.pictures_count,_source.bibliography,_source.applications_count,_source.pages_count,_source.sources_count,_source.books_count,_source.udk,_source.transmission_to_third_party_users,_source.number_of_views
0,dissertations,_doc,NUCPVIPPBEKK7LKTPWKF95M6,0.0,"[1657719740396, 0.0]",Научное и практическое обоснование получения э...,"[{'name': 'отходы ГПК и ТЭК'}, {'name': 'утили...",В работе приведены данные по утилизации отсево...,Докторская,Рукопись,Доктор технических наук,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,2536014538,Приморский край,25,ALEQVRVDWUZEG0AZ8HV7WSWM,d4ab74d492bdf940ff8d9bf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,ФГБОУ ВО ИРГУПС,1023801748761,3812010086,Иркутская обл,38,Валерий,Петухов,Иванович,Профессор,Доктор технических наук,Александр,Самардак,Сергеевич,Проректор по научной работе,"[{'name': 'Экология', 'code': '1.5.15.'}]",[{'name': 'Отходы производства и их использова...,"[{'name': 'MATERIALS SCIENCE, COMPOSITES', 'co...",2022-07-13T13:42:20.396027+00:00,522071300157-3,"[{'fio': 'Руш Е.А.', 'scientific_degree': {'na...","[{'fio': 'Шершнева М.В.', 'scientific_degree':...",2022-07-06,Коновалова,Наталия,Анатольевна,73,60,657,2,373,657,1,502.174.1 658.567 628.3/.4,False,2
1,dissertations,_doc,J6PJQH8V1SZT4YM64CSFEPJX,0.0,"[1657542275886, 0.0]",Цифровые системы связи и навигации в нестацион...,"[{'name': 'гидроакустическая связь'}, {'name':...",Современное освоение Мирового Океана неразрывн...,Докторская,Рукопись,Доктор технических наук,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,2536014538,Приморский край,25,JXDYN5VUB7LATD8HZ4740PRS,2e11cd14ccfce8a78646e73,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗОВ...,"ФГАОУ ВО ""ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ УНИВЕРСИ...",1022501297785,2536014538,Приморский край,25,Валерий,Петухов,Иванович,Профессор,Доктор технических наук,Александр,Самардак,Сергеевич,Проректор по научной работе,"[{'name': 'Физические поля корабля, океана, ат...","[{'name': 'Аппаратура передачи данных', 'code'...","[{'name': 'ACOUSTICS', 'code': '01.03.AA'}]",2022-07-11T12:24:35.886027+00:00,522071100113-1,"[{'fio': 'Стаценко Л.Г.', 'scientific_degree':...","[{'fio': 'Селезнев И.А.', 'scientific_degree':...",2022-06-23,Родионов,Александр,Юрьевич,27,123,317,7,352,317,1,621.384.3; 681.883,False,2
2,dissertations,_